In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from utils import *
import numpy as np
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
# datapath = '/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/UCSD/DSC/DSC180/ClimateBench - Plus/ClimateBench-Plus/DKL Gaussian Process/data/processed_data/'
datapath = 'G://My Drive//UCSD//DSC//DSC180//ClimateBench - Plus//ClimateBench-Plus//DKL Gaussian Process//data//processed_data//'
simulations = ['ssp126', 'ssp370', 'ssp585', 'hist-GHG', 'hist-aer']

In [3]:
X_train = []
Y_train = []

for i, simu in enumerate(simulations):
    input_name = 'inputs_' + simu + '.nc'
    output_name = 'outputs_' + simu + '.nc'
    # Just load hist data in these cases 'hist-GHG' and 'hist-aer'
    if 'hist' in simu:
        # load inputs 
        input_xr = xr.open_dataset(datapath + input_name)
            
        # load outputs                                                             
        output_xr = xr.open_dataset(datapath + output_name).mean(dim='member')
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400, "pr90": output_xr.pr90 * 86400})\
                             .rename({'lon':'longitude', 'lat': 'latitude'})\
                             .transpose('time','latitude', 'longitude').drop(['quantile'])
    
    # Concatenate with historical data in the case of scenario 'ssp126', 'ssp370' and 'ssp585'
    else:
        # load inputs 
        input_xr = xr.open_mfdataset([datapath + 'inputs_historical.nc', datapath + input_name]).compute()
            
        # load outputs                                                             
        output_xr = xr.concat([xr.open_dataset(datapath + 'outputs_historical.nc').mean(dim='member'),
                               xr.open_dataset(datapath + output_name).mean(dim='member')],
                               dim='time').compute()
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,"pr90": output_xr.pr90 * 86400})\
                             .rename({'lon':'longitude', 'lat': 'latitude'})\
                             .transpose('time','latitude', 'longitude').drop(['quantile'])

    print(input_xr.dims, simu)

    # Append to list 
    X_train.append(input_xr)
    Y_train.append(output_xr)

Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp126
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp370
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp585
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-GHG
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-aer


## Normaliza the Data

In [4]:
# Compute mean/std of each variable for the whole dataset
meanstd_inputs = {}
len_historical = 165

for var in ['CO2', 'CH4', 'SO2', 'BC']:
    # To not take the historical data into account several time we have to slice the scenario datasets
    # and only keep the historical data once (in the first ssp index 0 in the simus list)
    array = np.concatenate([X_train[i][var].data for i in [0, 3, 4]] + 
                           [X_train[i][var].sel(time=slice(len_historical, None)).data for i in range(1, 3)])
    print((array.mean(), array.std()))
    meanstd_inputs[var] = (array.mean(), array.std())

(1074.172303244536, 1755.690699230666)
(0.1927369743762821, 0.18457590641432994)
(2.5623359997066755e-12, 2.250114566783271e-11)
(1.4947905009818064e-13, 1.0313342554838387e-12)


In [5]:
# normalize input data 
X_train_norm = [] 
for i, train_xr in enumerate(X_train): 
    for var in ['CO2', 'CH4', 'SO2', 'BC']: 
        var_dims = train_xr[var].dims
        train_xr=train_xr.assign({var: (var_dims, normalize(train_xr[var].data, var, meanstd_inputs))}) 
    X_train_norm.append(train_xr)

#### For a given output variable (TAS) create the Train_X and Train_Y dataframes

In [6]:
var_to_predict =  'tas'
# skip_historical set to (i < 2) because of the order of the scenario and historical runs in the X_train and Y_train lists.
# In details: ssp126 0, ssp370 1 = skip historical part of the data, ssp585 2, hist-GHG 3 and hist-aer 4 = keep the whole sequence
X_train_all = np.concatenate([input_for_training(X_train_norm[i], skip_historical=(i<2), len_historical=len_historical) for i in range(len(simulations))], axis = 0)
Y_train_all = np.concatenate([output_for_training(Y_train[i], var_to_predict, skip_historical=(i<2), len_historical=len_historical) for i in range(len(simulations))], axis=0)
# add a dimension to the output data
Y_train_all = Y_train_all[..., np.newaxis]


X_train_all = X_train_all.reshape(726, 4, 96, 144)
Y_train_all = Y_train_all.reshape(726, 1, 96, 144)
X_train_all = torch.tensor(X_train_all).float()
Y_train_all = torch.tensor(Y_train_all).float()
print(X_train_all.shape)
print(Y_train_all.shape)

torch.Size([726, 4, 96, 144])
torch.Size([726, 1, 96, 144])


## Current output explainations

**X_train_all** : (726, 4, 96, 144)

- 726 samples (years of training data)
    - We are keeping each year independent of each other (no rolling window)
- 4 input variable - Aerosols (CO2, CH4, SO2, BC)
- 96 latitudes
- 144 longitudes

**Y_train_all** : (726, 1, 96, 144)

- 726 samples (years of training data)
    - We are keeping each year independent of each other (no rolling window)
- 1 output variable - TAS
    - *May add more output variables in the future*
- 96 latitudes
- 144 longitudes

## Prepping the data for Neural Process

1. First need to create a mask for the data (`create_context_target_mask` function in `utils.py`)
    - This will be used to mask out the missing data
    - Context Mask
        - `num_context_range` : Number of context points to be used *In this case it is the number of lat/lng pixels to be non-hidden*
    - Target Mask
        - `num_extra_target_range` : Number of extra points that is shown in the target set

In [7]:
test_batch_X = X_train_all[0:32]
test_batch_Y = Y_train_all[0:32]
print(test_batch_X.shape, test_batch_Y.shape)

torch.Size([32, 4, 96, 144]) torch.Size([32, 1, 96, 144])


In [8]:
def get_random_lat_lng_mask(context_point_shape, num_context, num_extra_target):
    aerosol_dim, lat_dim, lng_dim = context_point_shape
    
    # Empty mask
    context_mask = torch.zeros((aerosol_dim, lat_dim, lng_dim))
    target_mask = torch.zeros((aerosol_dim, lat_dim, lng_dim))

    # random lat and lng
    context_lat = np.random.randint(0, lat_dim, num_context)
    context_lng = np.random.randint(0, lng_dim, num_context)
    target_lat = np.random.randint(0, lat_dim, num_context + num_extra_target)
    target_lng = np.random.randint(0, lng_dim, num_context + num_extra_target)
    # set mask to 1
    context_mask[:, context_lat, context_lng] = 1
    target_mask[:, target_lat, target_lng] = 1
    
    return context_mask, target_mask

In [9]:
def create_context_target_mask(data_size, num_context, num_extra_target, batch_size, one_mask=True):
    aerosol_dim, lat, lng = data_size
    batch_context_mask = torch.zeros(batch_size, aerosol_dim, lat, lng)
    batch_target_mask = torch.zeros(batch_size, aerosol_dim, lat, lng)
    
    if one_mask:
        context_mask, target_mask = get_random_lat_lng_mask((1, lat, lng), num_context, num_extra_target)
        for i in range(batch_size):
            # apply the same mask to all the batch and all the aerosol variables
            for j in range(aerosol_dim):
                batch_context_mask[i, j] = context_mask
                batch_target_mask[i, j] = target_mask
        
        return batch_context_mask, batch_target_mask
    else:
        for i in range(batch_size):
            batch_context_mask[i], batch_target_mask[i] = get_random_lat_lng_mask((1, lat, lng), num_context, num_extra_target)
        
    return batch_context_mask, batch_target_mask

In [10]:
num_context_range = [2000, 8000]
num_extra_target_range = [1000, 5000]

num_context = np.random.randint(*num_context_range)
num_extra_target = np.random.randint(*num_extra_target_range)

data_size = (4, 96, 144)
batch_size = 32


batch_context_mask, batch_target_mask = create_context_target_mask(data_size, num_context, num_extra_target, batch_size)
print(batch_context_mask.shape, batch_target_mask.shape)

torch.Size([32, 4, 96, 144]) torch.Size([32, 4, 96, 144])


## Neural Process

### Encoder
Applies a CNN to the input data to get a representation of the input data (context points/target points)
- Reduces the input data to a fixed size representation (Batch_size, aerosols_dim + variable_dim, lat, lng) --> (Batch_size, representation_dim)
    - Representation_dim is set by user (In this case 128)
- Applies a ReLU activation function

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
class Encoder(nn.Module):
    """Maps an (x_i, y_i) pair to a representation r_i.

    Parameters
    ----------
    r_dim : int
        Dimension of output representation r.
    """
    def __init__(self, r_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=5, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(64, r_dim)

    def forward(self, context_points):
        """
        context_points : torch.Tensor
            Shape (batch_size, 4 (aerosols), 96 (lat), 144 (lon))
        """
        batch_size = context_points.shape[0]
        x_dim = context_points.shape[1:]
        # context_points shape: [batch_size, channels, height, width] -> [batch_size, 5, 96, 144]
        context_points = torch.relu(self.conv1(context_points))
        context_points = self.pool(context_points)
        context_points = torch.relu(self.conv2(context_points))
        context_points = self.pool(context_points)
        context_points = torch.relu(self.conv3(context_points))
        context_points = self.global_pool(context_points)  # Reduce to [batch_size, 64, 1, 1]
        context_points = torch.flatten(context_points, 1)  # Flatten to [batch_size, 64]
        context_points = self.fc1(context_points)  # Map to [batch_size, r_dim]
        return context_points

In [13]:
# prep the context points and target points
context_points_X = test_batch_X * batch_context_mask
context_points_Y = test_batch_Y * batch_context_mask[:, 0:1, :, :] # doesn't matter which aerosol variable mask we use as they are all the same

target_points_X = test_batch_X * batch_target_mask
target_points_Y = test_batch_Y * batch_target_mask[:, 0:1, :, :]

# get context and target points
context_points = torch.cat([context_points_X, context_points_Y], dim=1)
target_points = torch.cat([target_points_X, target_points_Y], dim=1)

context_points.shape

torch.Size([32, 5, 96, 144])

In [14]:
encoder = Encoder(r_dim=128)
r_i_context = encoder(context_points)
r_i_target = encoder(target_points)
print(r_i_context.shape, r_i_target.shape)

torch.Size([32, 128]) torch.Size([32, 128])


## Mu-Sigma Encoder
Takes the representation from the encoder and outputs the mean and variance of the distribution

In [15]:
class MuSigmaEncoder(nn.Module):
    """
    Maps a representation r to mu and sigma which will define the normal
    distribution from which we sample the latent variable z.

    Parameters
    ----------
    r_dim : int
        Dimension of output representation r.

    z_dim : int
        Dimension of latent variable z.
    """
    def __init__(self, r_dim, z_dim):
        super(MuSigmaEncoder, self).__init__()

        self.r_dim = r_dim
        self.z_dim = z_dim

        self.r_to_hidden = nn.Linear(r_dim, r_dim)
        self.hidden_to_mu = nn.Linear(r_dim, z_dim)
        self.hidden_to_sigma = nn.Linear(r_dim, z_dim)

    def forward(self, r):
        """
        r : torch.Tensor
            Shape (batch_size, r_dim)
        """
        hidden = torch.relu(self.r_to_hidden(r))
        mu = self.hidden_to_mu(hidden)
        # Define sigma following convention in "Empirical Evaluation of Neural
        # Process Objectives" and "Attentive Neural Processes"
        sigma = 0.1 + 0.9 * torch.sigmoid(self.hidden_to_sigma(hidden))
        return mu, sigma

In [16]:
# Setup mu and sigma encoder
r_dim, z_dim = 128, 64
mu_sigma_encoder = MuSigmaEncoder(r_dim, z_dim)

# Get mu and sigma
mu_context, sigma_context = mu_sigma_encoder(r_i_context)
mu_target, sigma_target = mu_sigma_encoder(r_i_target)
print("mu_context shape:", mu_context.shape)
print("sigma_context shape:", sigma_context.shape)
print("mu_target shape:", mu_target.shape)
print("sigma_target shape:", sigma_target.shape)

mu_context shape: torch.Size([32, 64])
sigma_context shape: torch.Size([32, 64])
mu_target shape: torch.Size([32, 64])
sigma_target shape: torch.Size([32, 64])


## Decoder

Takes the sampled latent representation from the mu-sigma encoder and the target points x-points to output the predicted distributions mu and sigma which can be used to calculate the loss and make predictions.

- xz_to_hidden : Takes the input pair (x, z context points) and outputs a hidden representation

In [17]:
class Decoder(nn.Module):
    """
    Maps target input x_target and samples z (encoding information about the
    context points) to predictions y_target.

    Parameters
    ----------
    """
    def __init__(self, h_dim, y_dim):
        super(Decoder, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=5, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(64, r_dim)
        self.relu = nn.ReLU(inplace=True)

    
        self.hidden_to_mu = nn.Linear(h_dim, y_dim)
        self.hidden_to_sigma = nn.Linear(h_dim, y_dim)
        
    def forward(self, xz_input_pair):
        """
        x : torch.Tensor
            Shape (batch_size, num_points, x_dim)

        z : torch.Tensor
            Shape (batch_size, z_dim)

        Returns
        -------
        Returns mu and sigma for output distribution. Both have shape
        (batch_size, num_points, y_dim).
        """
        xz_input_pair = torch.relu(self.conv1(xz_input_pair))
        xz_input_pair = self.pool(xz_input_pair)
        xz_input_pair = torch.relu(self.conv2(xz_input_pair))
        xz_input_pair = self.pool(xz_input_pair)
        xz_input_pair = torch.relu(self.conv3(xz_input_pair))
        xz_input_pair = self.global_pool(xz_input_pair)
        xz_input_pair = torch.flatten(xz_input_pair, 1)
        hidden = self.fc1(xz_input_pair)

        mu = self.hidden_to_mu(hidden)
        pre_sigma = self.hidden_to_sigma(hidden)

        
        # Define sigma following convention in "Empirical Evaluation of Neural
        # Process Objectives" and "Attentive Neural Processes"
        sigma = 0.1 + 0.9 * F.softplus(pre_sigma)
        return mu, sigma

In [18]:
# First we need to sample z from the distribution defined by mu and sigma
# We sample z for both context and target points


# Get context and target points Normal distributions
import torch.distributions as D

q_target = D.Normal(mu_target, sigma_target)
q_context = D.Normal(mu_context, sigma_context)

# Sample z
z_sample_target = q_target.rsample()

In [19]:
z_sample_target.shape

torch.Size([32, 64])

In [20]:
# Setup decoder
decoder = Decoder(128, 128)

# get the input for the decoder
xz_target = torch.cat([target_points_X, z_sample_target], dim=-1)
print(xz_target.shape)
# Get predictions
y_pred_mu, y_pred_sigma = decoder(xz_target)

RuntimeError: Tensors must have same number of dimensions: got 4 and 2